In [ ]:
#Invite everyone to the party
import pandas as pd
import numpy as np
import datetime as DT
import hmac
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine
import pyodbc 
from geopy import Point
from geopy.distance import distance



In [ ]:

#Connect to the database -- to be changed depending on configuration of the database

cnxn = pyodbc.connect(
    Trusted_Connection='Yes',
    Driver='{SQL Server Native Client 11.0}',
    Server='SERVER_NAME',
    Database='DATABASE_NAME'
)

cursor = cnxn.cursor()


In [ ]:
###Define customer dataframe
sql_customer = """SELECT [Customer_id]
      ,[Full_name]
      ,[Customer_type]
      ,[Is_client_flg]
      ,[Is_referal]
      ,[Ref_Customer_id]
      ,[Country]
      ,[Region]
      ,[City]
      ,[Street]
      ,[Street_nr]
      ,[Activation_date]
      ,[Contact_phone]
      ,[Contact_email]
      ,[Acc_mgr_id]
      ,[Acc_mgr_name]
  FROM [dbo].[Customer]"""

customer = pd.read_sql(sql_customer,cnxn)
customer.head()

customer['full_address'] = customer['Country'] + ","+ customer['Region'] + ","+ customer['City'] + ","+ customer['Street'] + ","+ customer['Street_nr'] + " " 

customer.head()



In [ ]:


###Cleaning the data
customer = customer.replace(to_replace='None', value=np.nan).dropna()



In [ ]:
#Geocode customer---takes a while
geolocator = Nominatim(user_agent="myApp")

for i in customer.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(customer['full_address'][i])
        
        #append lat/long to column using dataframe location
        customer.loc[i,'customer_lat'] = location.latitude
        customer.loc[i,'customer_long'] = location.longitude
        customer.loc[i,'customer_address'] = location.address
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        customer.loc[i,'customer_lat'] = ""
        customer.loc[i,'customer_long'] = ""
        customer.loc[i,'customer_address'] = ""

#print first rows as sample
customer.head()

In [ ]:
##Import this CSV into SQL database
customer.to_csv (r'file path', index = False) # place 'r' before the path name

